## Trec dataset 

In [17]:
retrieval_trec_2021 = "../trec_2021/retrieved_trials.json"
retrieval_trec_2022 = "../trec_2022/retrieved_trials.json"
retrieval_sigir = "../sigir/retrieved_trials.json"
import json
with open(retrieval_trec_2021, "r") as f1,open(retrieval_trec_2022, "r") as f2,open(retrieval_sigir, "r") as f3:
    trec_2021_query2note = json.load(f1)
    trec_2022_query2note = json.load(f2)
    sigir_query2note = json.load(f3)

# sigir_query2note
# [{'patient_id': 'sigir-20141',
#   'patient': 'A 58-year-old African-American woman presents to the ER with episodic pressing/burning anterior chest pain that began two days earlier for the first time in her life. The pain started while she was walking, radiates to the back, and is accompanied by nausea, diaphoresis and mild dyspnea, but is not increased on inspiration. The latest episode of pain ended half an hour prior to her arrival. She is known to have hypertension and obesity. She denies smoking, diabetes, hypercholesterolemia, or a family history of heart disease. She currently takes no medications. Physical examination is normal. The EKG shows nonspecific changes.',
#   '0': [{'brief_title': 'Usefulness of Chest Wall Tenderness as Bedside Test to Exclude Acute Coronary Syndrome in Different Demographic Groups',
#     'phase': '',
#     'drugs': "['Clinical examination: chest wall tenderness']",
#     'drugs_list': ['Clinical examination: chest wall tenderness'],
#     'diseases': "['Chest Pain']",
#     'diseases_list': ['Chest Pain'],
#     'enrollment': '110.0',
#     'inclusion_criteria': 'inclusion criteria: All patients over the age of 18 years presenting with the leading symptom of first time or recurrent acute chest pain in the emergency room of the Department of Internal Medicine, University Hospital of Zurich. \n\n ',
#     'exclusion_criteria': ': \n\n Missing informed consent. \n\n Cardiopulmonary unstable patients. \n\n No self reported chest pain. \n\n Recent thoracic surgery within1 year, inflammatory joint disease, fibromyalgia, cardiogenic shock.',
#     'brief_summary': 'To determine the significance of a simple bedside clinical test (chest wall tenderness) to exclude myocardial ischemia in different demographic groups.',
#     'NCTID': 'NCT01724996'},

In [19]:
# len(sigir_query2note) # 58
# len(trec_2021_query2note) #75
# len(trec_2022_query2note) # 50

## Detected the sensitive information 

In [20]:
from transformers import AutoModel, AutoTokenizer
import os 
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet

import re
# Only download once
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
sensitive_words = "../sensitive_list"
files = os.listdir(sensitive_words)
sensitive_list = []
for file in files:
    path = os.path.join(sensitive_words, file)
    with open(path, "r") as f:
        for i in f.readlines():
            sensitive_list.append(i.split("\n")[0])


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /jet/home/yji3/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /jet/home/yji3/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /jet/home/yji3/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [21]:
import copy

sensitive_categories = {
    'gender': {
        "he": "she",
        "she": "he",
        "his": "her",
        "her": "his",
        "him": "her",
        "man": "woman",
        "woman": "man",
        "boy": "girl",
        "girl": "boy",
        "male": "female",
        "female": "male",
        "husband": "wife",
        "wife": "husband",
    },
    'ethnicity': {
        "african-american": "white",
        "asian": "white",
        "hispanic": "white",
        "latino": "white",
        "latina": "white",
        "black": "white",
        # "white": "black",
        "caucasian": "black",
        "native american": "white",
        "indian": "white",
        "middle eastern": "white",
        "pacific islander": "white",
        "multiracial": "white",
        "biracial": "white",
        "jewish": "white",
        "arab": "white",
        "african": "white",
        "european": "black",
        "chinese": "white",
        "japanese": "white",
        "korean": "white",
        "filipino": "white",
        "vietnamese": "white",
        "thai": "white",
        "cambodian": "white",
        "lao": "white",
        "hmong": "white",
        "french": "white",
        "german": "white",
        "italian": "white",
        "irish": "white",
        "russian": "white",
        "polish": "white",
        "greek": "white",
        "portuguese": "white",
        "spanish": "white",
        "swedish": "white",
        "norwegian": "white",
        "dutch": "white",
        "belgian": "white",
        "swiss": "white",
        "austrian": "white",
        "danish": "white",
        "finnish": "white",
        "hungarian": "white",
        "romanian": "white",
        "bulgarian": "white",
        "yugoslav": "white",
        "slovak": "white",
        "czech": "white",
        "ukrainian": "white",
        "latvian": "white",
        "estonian": "white",
        "lithuanian": "white",
        "armenian": "white",
        "georgian": "white",
        "mongolian": "white",
        "turkish": "white",
        "persian": "white",
        "iraqi": "white",
        "syrian": "white",
        "lebanese": "white",
        "egyptian": "white",
        "moroccan": "white",
        "algerian": "white",
        "tunisian": "white",
        "libyan": "white",
        "ethiopian": "white",
        "somali": "white",
        "sudanese": "white",
        "kenyan": "white",
        "tanzanian": "white",
        "ugandan": "white",
        "rwandan": "white",
        "burundian": "white",
        "zambian": "white",
        "zimbabwean": "white",
        "namibian": "white",
        "botswanan": "white",
        "swazi": "white",
        "lesotho": "white",
        "mozambican": "white",
        "madagascan": "white",
        "mauritian": "white",
        "seychellois": "white",
        "australian": "white",
        "new zealander": "white",
        "papua new guinean": "white",
        "fijian": "white",
        "samoan": "white",
        "tongan": "white",
        "tuvaluan": "white",
        "kiribati": "white",
        "marshallese": "white",
        "micronesian": "white",
        "palauan": "white",
        "nauruan": "white",
        "vanuatuan": "white",
        "solomon islander": "white",
        "new caledonian": "white",
        "french polynesian": "white",
        "wallisian": "white",
        "futunan": "white",
        "american samoa": "white",
        "guamanian": "white",
        "northern mariana islander": "white",
        "hokkaido": "white",
        "okinawan": "white",
    
        # "african-american": "white",
        # "asian": "white",
        # "hispanic": "white"
        
    }
}


def get_replacement(word, category):
    if word.lower() in sensitive_categories[category]:
        return {sensitive_categories[category][word.lower()]}
    
    # antonyms = set()
    # for syn in wordnet.synsets(word):
    #     for lemma in syn.lemmas():
    #         if lemma.antonyms():
    #             antonyms.add(lemma.antonyms()[0].name())
    # return antonyms


def replace_with_replacements(phrase, category):
    words = phrase.split()
    replaced_words = []
    for word in words:
        replacements = get_replacement(word, category)
        if replacements:
            replaced_words.append(list(replacements)[0])
        else:
            replaced_words.append(word)
    return ' '.join(replaced_words)


def replace_sensitive_phrases(text, sensitive_phrases, category):
    for phrase in sorted(sensitive_phrases, key=len, reverse=True):
        replacement_phrase = replace_with_replacements(phrase, category)
        text = re.sub(r'\b' + re.escape(phrase) + r'\b', replacement_phrase, text, flags=re.IGNORECASE)
    return text


def classify_sensitive_phrases(sensitive_phrases):
    classified_phrases = {'gender': [], 'ethnicity': []}
    for phrase in sensitive_phrases:
        tokens = phrase.split()
        for token in tokens:
            for category in sensitive_categories:
                if token.lower() in sensitive_categories[category]:
                    # print("token", token)
                    # print("phrase", phrase)
                    classified_phrases[category].append(token)
                    break
    return classified_phrases


def find_sensitive_phrases(tokens, sensitive_words):
    detected_sensitive_words = set()
    token_count = len(tokens)
    
    lower_sensitive_words = set([word.lower() for word in sensitive_words])


    max_window_size = 2
    
    for window_size in range(1, max_window_size + 1):
        for i in range(token_count - window_size + 1):
            window_tokens = tokens[i:i + window_size]
            window_phrase = " ".join(window_tokens).lower()
            

            if any(word in window_phrase.split() for word in lower_sensitive_words):
                detected_sensitive_words.add(window_phrase)
    
    return list(detected_sensitive_words)

def replace_all_sensitive_categories(patient_note, classified_phrases, categories_to_replace=None):
    new_patient_note = copy.deepcopy(patient_note)
    if categories_to_replace is None:
        categories_to_replace = sensitive_categories.keys()
    
    for category in categories_to_replace:
        new_patient_note = replace_sensitive_phrases(new_patient_note, classified_phrases[category], category)
        # print("patient_note inside", new_patient_note)
    return new_patient_note


res = {}
for query2note in sigir_query2note:
    patient_id = query2note["patient_id"]
    res[patient_id] = {}
    
    patient_note = query2note["patient"]
    res[patient_id]["patient"] = patient_note
    clinical_trials = query2note["patient"]
    tokens = word_tokenize(patient_note)
    sensitive = find_sensitive_phrases(tokens,sensitive_list) # ['she', 'her', 'in her life', '58-year-old african-american woman', 'to her arrival',...]
    classified_phrases = classify_sensitive_phrases(sensitive)
    categories_to_replace = ['gender', 'ethnicity']
    
    new_text = replace_all_sensitive_categories(patient_note, classified_phrases, categories_to_replace)
    res[patient_id]["gender"] = replace_all_sensitive_categories(patient_note, classified_phrases, ["gender"]) if classified_phrases["gender"]!=[] else None
    res[patient_id]["ethnicity"] = replace_all_sensitive_categories(patient_note, classified_phrases, ["ethnicity"]) if classified_phrases["ethnicity"]!=[] else None
    res[patient_id]["all"] = replace_all_sensitive_categories(patient_note, classified_phrases,categories_to_replace)


In [22]:
import json
with open("sigir_sensitive.json", "w") as f:
    json.dump(res, f)

In [24]:
res = {}
for query2note in trec_2022_query2note:
    patient_id = query2note["patient_id"]
    res[patient_id] = {}
    
    patient_note = query2note["patient"]
    res[patient_id]["patient"] = patient_note
    clinical_trials = query2note["patient"]
    tokens = word_tokenize(patient_note)
    sensitive = find_sensitive_phrases(tokens,sensitive_list) # ['she', 'her', 'in her life', '58-year-old african-american woman', 'to her arrival',...]
    classified_phrases = classify_sensitive_phrases(sensitive)
    categories_to_replace = ['gender', 'ethnicity']
    
    new_text = replace_all_sensitive_categories(patient_note, classified_phrases, categories_to_replace)
    res[patient_id]["gender"] = replace_all_sensitive_categories(patient_note, classified_phrases, ["gender"]) if classified_phrases["gender"]!=[] else None
    res[patient_id]["ethnicity"] = replace_all_sensitive_categories(patient_note, classified_phrases, ["ethnicity"]) if classified_phrases["ethnicity"]!=[] else None
    res[patient_id]["all"] = replace_all_sensitive_categories(patient_note, classified_phrases,categories_to_replace)
import json
with open("trec_2022_sensitive.json", "w") as f:
    json.dump(res, f)

## Use TrialGPT to verify 

In [1]:
# with the already defined three already defined dataset
# Add openai key
with open("OpenaiKey.txt", "r")as f:
    item = f.read()
item = item.strip()
from openai import OpenAI

import os
os.environ["OPENAI_API_KEY"] = item
client = OpenAI()
# response = client.chat.completions.create(
#   model="gpt-4",
#   messages=[
#     {"role": "system", "content": "You are a helpful assistant."},
#     {"role": "user", "content": "Who won the world series in 2020?"},
#     {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
#     {"role": "user", "content": "Where was it played?"}
#   ]
# )
# response.choices[0].message.content

In [ ]:
from TrialGPT import trialgpt_matching
import json
from nltk.tokenize import sent_tokenize
import os
import sys
corpuses = ["sigir", "trec_2021", "trec_2022"]
model = "gpt-4"
def process_patient(patient):
    sents = sent_tokenize(patient)
    sents.append("The patient will provide informed consent, and will comply with the trial protocol without any practical issues.")
    sents = [f"{idx}. {sent}" for idx, sent in enumerate(sents)]
    patient = "\n".join(sents)
    return patient
for corpus in corpuses:
    original_dataset = json.load(open(f"../{corpus}/retrieved_trials.json"))
    
    new_patient_note = json.load(open(f"{corpus}_sensitive.json"))

    output_path = f"fairness_results/matching_results_{corpus}_{model}.json" 

    # Dict{Str(patient_id): Dict{Str(label): Dict{Str(trial_id): Str(output)}}}
    if os.path.exists(output_path):
        output = json.load(open(output_path))
    else:
        output = {}
         
        

    for instance in original_dataset[:20]:  # here I limit every label only using 20
        # Dict{'patient': Str(patient), '0': Str(NCTID), ...}
        patient_id = instance["patient_id"]
        patient = instance["patient"]
        sensitive = new_patient_note[patient_id]
        cat_sensitive = {}
        for k, v in sensitive.items():
            if v!=None and k!="patient":
                cat_sensitive[k] = process_patient(v)
        # same process 
        original_patient = process_patient(patient)
        
        

        # initialize the patient id in the output 
        if patient_id not in output:
            output[patient_id] = {}
            for sensitive_type, _ in cat_sensitive.items():
                output[patient_id][sensitive_type] = {"0": {}, "1": {}, "2": {}}
        
        
        # only use the sensitive changed as the input patient note
        for sensitive_type, sensitive_note in cat_sensitive.items():
            for label in ["2", "1", "0"]:
                if label not in instance: continue
                
                for trial in instance[label][:10]:  # here I limit every label only using 30
                    trial_id = trial["NCTID"]
                    
                    # already calculated and cached
                    if trial_id in output[patient_id][sensitive_type][label]:
                        continue

                    # in case anything goes wrong (e.g., API calling errors)
                    try:
                        results = trialgpt_matching(trial, sensitive_note, model, client)
                        # results = "sdsa"
                        output[patient_id][sensitive_type][label][trial_id] = results
                        # output[patient_id][]
                        with open(output_path, "w") as f:
                            json.dump(output, f, indent=4)

                    except Exception as e:
                        print(e)
                        continue